<a href="https://colab.research.google.com/github/atmark-techno/google-colab-notebooks/blob/master/Armadillo_IoT_G4_create_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに

本ドキュメントではArmadillo-IoT ゲートウェイ G4上で使用できるTFLite形式の推論モデルを、既存のモデルをベースに学習する転移学習を行い作成するサンプルを紹介します。

「Armadillo Base OS開発ガイド」の「5.1.1 教師データの用意」まで完了していることを前提としています。

# 準備

## Google Driveのマウント

最終的に作成するTFLite形式のファイルと、それを生成する元となるSavedModel形式のファイルはGoogle Driveに保存しますので、最初にGoogle Driveをマウントします。

In [ ]:
from google.colab import drive
drive.mount('./Drive')

## TensorFlowのインストール

In [ ]:
!pip install -U --pre tensorflow=="2.7.0"

## modelsリポジトリのクローン

TensorFlow公式が提供しているmodelsリポジトリをクローンします。

In [ ]:
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

## Object Detection APIのインストール

modelsリポジトリに含まれるObject Detection APIをインストールします。

In [ ]:
%%bash
cd models/research
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install --use-feature=2020-resolver .

Object Detection APIが正しくインストールされたかテストします。

In [ ]:
!python models/research/object_detection/builders/model_builder_tf2_test.py

## ライブラリ群のインポート

以後必要なライブラリをまとめてインポートします。

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import shutil
import copy
import cv2
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.utils import config_util
from object_detection.builders import model_builder

%matplotlib inline

## ディレクトリの生成

作業用のディレクトリを生成します。
作業用ディレクトリは、Google Colaboratoryのランタイムが終了すると自動的に消去されます。

In [ ]:
%%bash
# 作業用ディレクトリ
mkdir -p working/01_train_data
mkdir -p working/02_annotations
mkdir -p working/03_pretrained_model

## 教師データの用意

Armadillo Base OS開発ガイドの「5.1. 推論モデルの作成」の手順に従って作成した教師データを、それぞれ本Colab内の所定のディレクトリにアップロードしてください。

アップロードは画面左のファイルツリーからアップロードできます。

![アップロード方法](https://download.atmark-techno.com/armadillo-guide-std/document/images/howto_upload.png)

1. 画像ファイル(ATDEの~/datset_gauge/downloads/gauge/*.jpg)  
  /content/working/01_train_data/にアップロード  
  
  ![画像ファイルのアップロード](https://download.atmark-techno.com/armadillo-guide-std/document/images/upload_images.png)
2. .xmlファイル及び、tf_label_map.pbtxt(ATDEの~/dataset_gauge/annotations)  
  /content/working/02_annotations/にアップロード  
  
  ![画像ファイルのアップロード](https://download.atmark-techno.com/armadillo-guide-std/document/images/upload_annotations.png)


# 転移学習

ここからは既存の推論モデルをベースに、検出したい物体が検出できるようにチューニングを行う、転移学習の具体的な手順の一例を紹介します。

本手順では物体検出の転移学習についてのみ取り扱います。
物体検出以外の推論モデルにつきましては、大きく手順が異なることがありますので、適宜公式のマニュアル等を参照してください。

## ベースとなる推論モデルをダウンロード

本ドキュメントでは、[COCOデータセット](https://cocodataset.org/#explore)でトレーニング済みのSSD MobileNet V2をベースモデルとします。

ベースモデルを[TensorFlow2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)からダウンロード・展開します。

In [ ]:
%%bash
cd working/03_pretrained_model/
wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
tar xf ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
rm ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

## 教師データの分割

用意した教師データを学習用データ75%、検証用データ25%の割合で分けます。

In [ ]:
# 各ディレクトリ名を定義
images_dir = '/content/working/01_train_data'
annotations_dir = '/content/working/02_annotations'

train_dir = '/content/working/train'
train_images_dir = '/content/working/train/gauge/JPEGImages'
train_annotations_dir = '/content/working/train/VOC2007/Annotations'

val_dir = '/content/working/val'
val_images_dir = '/content/working/val/gauge/JPEGImages'
val_annotations_dir = '/content/working/val/VOC2007/Annotations'

In [ ]:
# 定義したディレクトリを作成
!mkdir -p $train_images_dir
!mkdir -p $train_annotations_dir
!mkdir -p $train_dir/VOC2007/ImageSets/Main

!mkdir -p $val_images_dir
!mkdir -p $val_annotations_dir
!mkdir -p $val_dir/VOC2007/ImageSets/Main

In [ ]:
# ファイル数をカウント
file_count = len(glob.glob(annotations_dir + '/*.xml'))
print('File count : ' + str(file_count))

# 学習データと検証データに分割
train_ratio = 0.75

file_list = glob.glob(annotations_dir + '/*.xml')
random_sample_list = random.sample(file_list, file_count)

# ディレクトリへコピー
for index, filepath in enumerate(random_sample_list):
  imagepath = os.path.join(images_dir, os.path.splitext(os.path.basename(filepath))[0] + '.jpg')
  if index < int(file_count * train_ratio):
    # 学習データとしてコピー
    shutil.copy2(filepath, train_annotations_dir)
    shutil.copy2(imagepath, train_images_dir)
  else:
    # 検証データとしてコピー
    shutil.copy2(filepath, val_annotations_dir)
    shutil.copy2(imagepath, val_images_dir)

## tfrecord形式に変換

分けた教師データをそれぞれtfrecord形式に変換します。

tfrecordとは、TensorFlowが推奨するTensorFlowでデータを学習させる際のフォーマットです。
詳細は[公式ドキュメント](https://www.tensorflow.org/tutorials/load_data/tfrecord?hl=ja)をご確認ください。

TensorFlowはもちろんcsvやxmlなどの他のフォーマットからでも学習可能ですが、tfrecord形式にすることで、メモリに収まりきらないような大規模なデータセットを扱うことができます。

Object Detection APIには、PascalVOC形式のxmlアノテーションファイルをtfrecordに変換するスクリプトが付属しています。
今回はこちらをそのまま利用するために、スクリプトに合わせたディレクトリ構成を構築します。


In [ ]:
!ls /content/working/train/VOC2007/Annotations/*.xml | xargs -i basename {} .xml > /content/working/train/VOC2007/ImageSets/Main/aeroplane_train.txt
!ls /content/working/val/VOC2007/Annotations/*.xml | xargs -i basename {} .xml > /content/working/val/VOC2007/ImageSets/Main/aeroplane_val.txt

In [ ]:
# 学習用データセットをtfrecord化
%%bash
python models/research/object_detection/dataset_tools/create_pascal_tf_record.py \
--data_dir=/content/working/train \
--year=VOC2007 \
--output_path=/content/working/train.record \
--label_map_path=/content/working/02_annotations/label_map.pbtxt

In [ ]:
# 検証用データセットをtfrecord化
%%bash
python models/research/object_detection/dataset_tools/create_pascal_tf_record.py \
--data_dir=/content/working/val \
--set=val \
--year=VOC2007 \
--output_path=/content/working/eval.record \
--label_map_path=/content/working/02_annotations/label_map.pbtxt

## pipeline.configの修正

/content/working/03_pretrained_model/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.configを修正します。

上記のpipeline.configへのリンクをクリックすると、Google Colaboratory上で編集することが可能です。

以下の箇所を修正してください。

* 3行目: num_classes: 90 → 1
 * 検出する物の種類数です。
 * 今回は"gauge"のみなので1とします。
* 138行目: batch_size: 512 → 16
 * 確率的勾配降下法を使用する場合、異常値の影響を小さくするためにデータセットをいくつかのサブセットに分けて学習します。
 * そのサブセットに含まれるデータの数を定義します。
 * 大きくすると学習時の負荷は高まり、環境によってはメモリ不足でエラーとなることがあります。
 * 2のn乗の数値を設定してください。
* 162行目: fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED" → "/content/working/03_pretrained_model/ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ckpt-0"
 * 転移学習のベースモデルを指定します。
 * "ckpt-0"までを記載すれば大丈夫です。
* 168行目: fine_tune_checkpoint_type: "classification" → "detection"
 * このパラメータによって、学習済みモデルのどの層を学習し直すかを選択できます。
 * このパラメータは、以下の3つから選べます。
   * "classification"  
   学習済みモデルから、分類バックボーン部分以外を学習し直します。主に、学習済み画像分類モデルから物体検出モデルを学習する際によく使用されます。
   * "detection"  
   学習済みモデルから、ボックス予測ヘッドとクラス予測ヘッドを学習し直します。主に、学習済み物体検出モデルに新たにクラスを追加する際によく使用されます。
   * "full"  
   学習済みモデルから、全てのパラメータを学習し直します。今回は使用しませんが、取り扱いが複雑なパラメータなので使用する際には注意してください。
  * 今回は学習済み物体検出モデルに新たに"gauge"クラスを追加したいので、"detection"に設定します。
* 172行目: label_map_path: "PATH_TO_BE_CONFIGURED" → "/content/working/02_annotations/label_map.pbtxt"
 * 学習用にラベルマップファイル(label_map.pbtxt)のパスを指定します。
* 174行目: input_path: "PATH_TO_BE_CONFIGURED" → "/content/working/train.record"
 * 学習用データ(train.record)のパスを指定します。
* 182行目: label_map_path: "PATH_TO_BE_CONFIGURED" → "/content/working/02_annotations/label_map.pbtxt"
 * 検証用にラベルマップファイル(label_map.pbtxt)のパスを指定します。
* 186行目: input_path: "PATH_TO_BE_CONFIGURED" → "/content/working/eval.record"
 * 検証用データ(train.record)のパスを指定します。

編集後は、Ctrl+Sで保存できます。

pipeline.configについては[公式ドキュメント](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/configuring_jobs.md)も合わせて参照してください。

## 学習結果保存用ディレクトリ作成

Google Drive内にArmadillo-BaseOS-Guideという名前で、転移学習後の推論モデルを保存するディレクトリを作成します。

既に同名のディレクトリが存在すると消去して作り直すので注意してください。

In [ ]:
!rm -rf Drive/MyDrive/Armadillo-BaseOS-Guide/
!mkdir -p Drive/MyDrive/Armadillo-BaseOS-Guide/

## TensorBoard

TensorFlowには、学習中の精度などのパラメータの推移を可視化するTensorBoardという機能があります。詳細は[公式ドキュメント](https://www.tensorflow.org/tensorboard?hl=ja)を参照してください。

Google Colaboratory上でも使用可能ですので使ってみます。

In [ ]:
%load_ext tensorboard

In [ ]:
tensorboard --logdir '/content/Drive/MyDrive/Armadillo-BaseOS-Guide'

## 学習開始

いよいよ転移学習を開始します。  
ここまで準備してきたので、学習及び検証はObject Detection APIで用意されているPythonスクリプトに必要な情報を渡すだけで実行できます。

大量のログが出力された後に、各ステップ毎の学習結果が出力され始めたら転移学習が正しく開始されたことになります。

直前のセルで実行したTensorBoardで学習中の進捗が確認できます。

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path="/content/working/03_pretrained_model/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config" \
    --model_dir="/content/Drive/MyDrive/Armadillo-BaseOS-Guide" \
    --num_train_steps=1000 \
    --alsologtostderr \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps=100

# TFLite形式への変換

ここまでの手順で作成した推移モデルは、Armadilloのようなエッジデバイス向けのモデルではありません。

以下のスクリプトを実行することで、エッジデバイス向けに最適化されたTFLite形式の推移モデルに変換することができます。

## 変換の前処理

転移学習で得た推移モデルは直接TFLite形式に変換することはできず、まずはTFLite形式への変換に適したSavedModel形式に変換します。

In [ ]:
%%bash
python models/research/object_detection/export_tflite_graph_tf2.py \
  --pipeline_config_path "/content/working/03_pretrained_model/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config" \
  --trained_checkpoint_dir "/content/Drive/MyDrive/Armadillo-BaseOS-Guide" \
  --output_directory "/content/Drive/MyDrive/Armadillo-BaseOS-Guide/tflite"

## 変換の実行

前処理が完了したら、TFLite形式への変換を行います。

ここでは単純に変換するのではなく、Armadillo-IoT G4搭載のNPU向けに整数量子化を行います。
量子化を行うことでわずかにモデルの精度が低下しますが、モデルのサイズの削減や推論速度の向上、メモリ使用量の削減などが期待できます。

モデルの量子化について、詳細は[公式ドキュメント](https://www.tensorflow.org/lite/performance/post_training_quantization?hl=ja)を合わせて参照してください。



In [ ]:
def representative_dataset():
  dataset_list = tf.data.Dataset.list_files('/content/working/01_train_data/*.jpg')
  for i in range(100): # ここの値は用意した画像数によって上下する(100~500程度が良いとされている)
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image / 255, tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/Drive/MyDrive/Armadillo-BaseOS-Guide/tflite/saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model = converter.convert()

## 変換後の推論モデルの保存

model.tfliteというファイル名で保存します。

In [ ]:
open('/content/Drive/MyDrive/Armadillo-BaseOS-Guide/tflite/model.tflite', 'wb').write(tflite_quant_model)

# 生成したtfliteモデルの動作確認

保存したmodel.tfliteを読み出し、実際に物体検出を行ってみます。

以下の手順は、最終的なArmadillo-IoT ゲートウェイ G4上でmodel.tfliteを使用したアプリケーションを作成する際の推論モデルの取り扱いの参考としても使用できます。

## model.tfliteの読み出し

In [ ]:
interpreter = tf.lite.Interpreter(model_path='/content/Drive/MyDrive/Armadillo-BaseOS-Guide/tflite/model.tflite')
interpreter.allocate_tensors()

## TFLite形式のモデルから入出力情報を読み取る

In [ ]:
# get_[input|output]_detailsで推論モデルの入出力情報を得ることができる
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 入力画像サイズを取得
_, input_height, input_width, _ = input_details[0]['shape']

## 推論

In [ ]:
# 適当な画像をリサイズして入力
imagefile = '/content/working/01_train_data/000001.jpg'
input_data = np.array(Image.open(imagefile).resize((input_width, input_height), Image.BICUBIC))
input_data = np.expand_dims(input_data, 0)
interpreter.set_tensor(input_details[0]['index'], input_data.astype(np.uint8))

# 推論の実行
interpreter.invoke()

# 推論結果を取得して表示
scores = interpreter.get_tensor(output_details[0]['index'])[0]
boxes = interpreter.get_tensor(output_details[1]['index'])[0]
print(scores)
print(boxes)

# 推論結果にしきい値を設定して抽出・整形
threshold = 240
results = []
for i in range(len(scores)):
  if scores[i] >= threshold:
    res = {}
    res['score'] = scores[i]
    res['box'] = boxes[i]
    results.append(res)
print(results)

## 推論結果の描画

In [ ]:
# 元の画像をOpenCVで読み出し
image = cv2.imread(imagefile, cv2.IMREAD_UNCHANGED)
image_height, image_width, _ = image.shape
cp = copy.deepcopy(image)

# 推論結果を整形して元画像の上に描画
for result in results:
  y1, x1, y2, x2 = result['box']
  x1 = int(x1 * (image_width / input_width))
  x2 = int(x2 * (image_width / input_width))
  y1 = int(y1 * (image_height / input_height))
  y2 = int(y2 * (image_height / input_height))
  score = result['score']
  cv2.rectangle(cp, (x1, y1), (x2, y2), (255, 0, 0), 2)
  cv2.putText(cp, str('{:.2f}'.format(score)), (x1, y1-10), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2, cv2.LINE_AA)

# test.jpgとして保存
cv2.imwrite('test.jpg', cp)

保存したtest.jpgを開き、推論結果を確認してください。

# 推論モデルのダウンロード

以下を実行することで、model.tfliteをローカルにダウンロードできます。


In [ ]:
from google.colab import files
files.download('/content/Drive/MyDrive/dist/tflite/model.tflite') 

以上で推論モデルの生成は完了です。

Armadillo-IoT ゲートウェイ G4への組み込み、アプリケーション開発については、引き続き「Armadillo Base OS 開発ガイド」を参照してください。